In [15]:
PROJECT= ! (gcloud config get-value project) 
BUCKET_NAME = PROJECT[0] + "-bucket"
INPUT_NAME="data.jsonl"
OUTPUT_DIR="results"
OUTPUT_FILE="results.jsonl"

In [18]:
prompt = """
Service Rep: How may I assist you today?
Customer: I need to change the shipping address for an order.
Service Rep: Ok, I can help you with that if the order has not been fulfilled from our warehouse yet. But if it has already shipped, then you will need to contact the shipping provider. Do you have the order ID?
Customer: Yes, it's 88986367.
Service Rep: One minute please while I pull up your order information.
Customer: No problem
Service Rep: Ok, it looks like your order was shipped from our warehouse 2 days ago. It is now in the hands of  the shipping provider, so you will need to contact them to update your delivery details. You can track your order with the shipping provider here: https://www.shippingprovider.com
Customer: Sigh, ok.
Service Rep: Is there anything else I can help you with today?
Customer: No, thanks.

Can you help me categorize the conversation above?

Format your response in the following format - 

clues: <clues> 

issue: <issue> 

resolution: <resolution> 

category: <category> 

sub-category: <sub-category> """

# Let's try Online Prediction...

In [39]:
from vertexai.preview.language_models import TextGenerationModel
text_model = TextGenerationModel.from_pretrained("text-bison")

parameters={
      "max_output_tokens": 200,
      "temperature": 0,
      "top_p": 0.95,
      "top_k": 40,
}

response = text_model.predict(
    prompt,
    **parameters,
)
print(f"Response from Model: \n\n{response.text}")

Response from Model: 

 clues: change shipping address, order ID: 88986367

issue: customer wants to change the shipping address for an order

resolution: the order has already shipped, so the customer will need to contact the shipping provider to update their delivery details

category: order

sub-category: change shipping address


### try a few more times (pretty consistent)

In [86]:
response = text_model.predict(
    prompt,
    **parameters,
)
print(f"Response from Model: \n\n{response.text}")

Response from Model: 

 clues: change shipping address, order ID: 88986367

issue: customer wants to change the shipping address for an order

resolution: the order has already shipped, so the customer will need to contact the shipping provider to update their delivery details

category: order

sub-category: change shipping address


In [87]:
response = text_model.predict(
    prompt,
    **parameters,
)
print(f"Response from Model: \n\n{response.text}")

Response from Model: 

 clues: change shipping address, order ID: 88986367

issue: customer wants to change the shipping address for an order

resolution: the order has already shipped, so the customer will need to contact the shipping provider to update their delivery details

category: order

sub-category: change shipping address


# Now let's try Batch Prediction

using same prompt a few times to see consistency

In [88]:
import json
import os
from google.cloud import storage

input_text = [{"prompt":prompt},{"prompt":prompt},{"prompt":prompt},{"prompt":prompt},{"prompt":prompt}]

with open(INPUT_NAME, 'w') as f:
    for item in input_text:
        f.write(json.dumps(item) + "\n")
        
# Get the bucket name
bucket_name = BUCKET_NAME

# Get the file path
file_path = INPUT_NAME

# Create a storage client
client = storage.Client()
bucket = client.bucket(bucket_name)

# Upload the file
bucket.blob(file_path).upload_from_filename(file_path)

In [89]:

batch_prediction_job = text_model.batch_predict(
  dataset=["gs://"+BUCKET_NAME+"/"+INPUT_NAME],
  destination_uri_prefix="gs://"+BUCKET_NAME+"/"+OUTPUT_DIR,
  # Optional:
  model_parameters={
      "maxOutputTokens": "200",
      "temperature": "0",
      "topP": "0.95",
      "topK": "40",
  },
)
print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/185246287903/locations/us-central1/batchPredictionJobs/7494703706588315648
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/185246287903/locations/us-central1/batchPredictionJobs/7494703706588315648')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/7494703706588315648?project=185246287903
BatchPredictionJob projects/185246287903/locations/us-central1/batchPredictionJobs/7494703706588315648 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/185246287903/locations/us-central1/batchPredictionJobs/7494703706588315648 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/185246287903/locations/us-central1/batchPredictionJobs/7494703706588315648 current state:
JobState.JOB_STATE_SUCCEEDED
BatchPredictionJob run completed. Resource name: projects/185246287903/locations/

In [90]:
blobs = bucket.list_blobs(prefix=batch_prediction_job.output_info.gcs_output_directory.replace("gs://"+BUCKET_NAME+"/",""))

for blob in blobs:
    print(blob.name)
    blob.download_to_filename(OUTPUT_FILE)

results/prediction-model-2023-09-29T19:30:47.895290Z/000000000000.jsonl


In [91]:
import json

batch_response = []

with open(OUTPUT_FILE, 'r') as f:
    for row in f:
        batch_response.append(json.loads(row))

In [93]:
for resp in batch_response: 
    print("\n\nRESPONSE: \n\n")
    print(resp["predictions"][0]["content"])



RESPONSE: 


clues: customer wants to change shipping address, order has already shipped

issue: change shipping address

resolution: customer needs to contact shipping provider

category: customer service

sub-category: order management


RESPONSE: 


clues: customer wants to change shipping address, order has already shipped

issue: change shipping address

resolution: customer needs to contact shipping provider

category: customer service

sub-category: order management


RESPONSE: 


Clues:
- Customer wants to change shipping address
- Order has already shipped
- Customer needs to contact shipping provider

Issue:
- Customer wants to change shipping address for an order that has already shipped

Resolution:
- Customer is told to contact shipping provider

Category:
- Customer Service

Sub-category:
- Order Management


RESPONSE: 


Clues:
- Customer wants to change shipping address
- Order has already shipped
- Customer needs to contact shipping provider

Issue:
- Customer wants 